## Random Uniform Grid Sampling

In [24]:
import jax
import jax.numpy as jnp
import jax.random as jr
import numpy as np
import platform
import equinox as eqx
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import qmc  # for Latin Hypercube Sampling



## BACKEND AUTOCONFIG - find GPU if it's there
print("\nconfiguring backend...")
system = platform.system()
machine = platform.machine().lower()

if system == "Darwin" and ("arm" in machine or "apple" in machine or "m1" in machine or "m2" in machine):
    try:
        jax.config.update("jax_platform_name", "METAL")
        print("Configured JAX backend: metal (Apple Silicon)")
    except Exception as e:
        print("Metal not available, falling back to default:", e)
elif system == "Linux":
    devices = jax.devices()
    if any(d.platform == "gpu" for d in devices):
        jax.config.update("jax_platform_name", "gpu")
        print("Configured JAX backend: gpu")
    else:
        jax.config.update("jax_platform_name", "cpu")
        print("Configured JAX backend: cpu")
else:
    jax.config.update("jax_platform_name", "cpu")
    print("Configured JAX backend: cpu")

print("backend selected:\n", jax.default_backend())
print("active devices:\n", jax.devices())
print("--------------------\n")



configuring backend...
Configured JAX backend: metal (Apple Silicon)
backend selected:
 METAL
active devices:
 [METAL(id=0)]
--------------------



In [16]:
n_bc = 200 # number of boundary pts
n_ic = 100 # num of init pts
n_re = 2000 # num residual pts
t_min, t_max = 0, 1
x_min, x_max = -1,1

x_re = np.random.uniform(x_min, x_max, size=(n_re, 1))
t_re = np.random.uniform(t_min, t_max, size=(n_re, 1))
xt_re = np.hstack([x_re, t_re])


x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2, ]) # two boundary halves
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc) # generate coords on boundaries
xt_bc = np.concatenate([np.reshape(xv_bc, [-1,1]), np.reshape(tv_bc, [-1,1])], axis = 1) # reshape to [(xi, ti)]


x_ic = np.random.uniform(x_min, x_max, [n_ic,])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.concatenate([np.reshape(xv_ic,[-1,1]), np.reshape(tv_ic,[-1,1])],axis=1)
                       
print(xt_re.shape)
print(xt_bc.shape)
print(xt_ic.shape)

np.savez(
    "data/uniform_2000.npz",
    xt_bc=xt_bc,
    xt_ic=xt_ic,
    xt_re=xt_re
)

(2000, 2)
(200, 2)
(100, 2)


In [17]:
n_bc = 200 # number of boundary pts
n_ic = 100 # num of init pts
n_re = 4000 # num residual pts
t_min, t_max = 0, 1
x_min, x_max = -1,1

x_re = np.random.uniform(x_min, x_max, size=(n_re, 1))
t_re = np.random.uniform(t_min, t_max, size=(n_re, 1))
xt_re = np.hstack([x_re, t_re])


x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2, ]) # two boundary halves
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc) # generate coords on boundaries
xt_bc = np.concatenate([np.reshape(xv_bc, [-1,1]), np.reshape(tv_bc, [-1,1])], axis = 1) # reshape to [(xi, ti)]


x_ic = np.random.uniform(x_min, x_max, [n_ic,])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.concatenate([np.reshape(xv_ic,[-1,1]), np.reshape(tv_ic,[-1,1])],axis=1)
                       
print(xt_re.shape)
print(xt_bc.shape)
print(xt_ic.shape)

np.savez(
    "data/uniform_4000.npz",
    xt_bc=xt_bc,
    xt_ic=xt_ic,
    xt_re=xt_re
)

(4000, 2)
(200, 2)
(100, 2)


In [18]:
n_bc = 200 # number of boundary pts
n_ic = 100 # num of init pts
n_re = 6000 # num residual pts
t_min, t_max = 0, 1
x_min, x_max = -1,1

x_re = np.random.uniform(x_min, x_max, size=(n_re, 1))
t_re = np.random.uniform(t_min, t_max, size=(n_re, 1))
xt_re = np.hstack([x_re, t_re])


x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2, ]) # two boundary halves
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc) # generate coords on boundaries
xt_bc = np.concatenate([np.reshape(xv_bc, [-1,1]), np.reshape(tv_bc, [-1,1])], axis = 1) # reshape to [(xi, ti)]


x_ic = np.random.uniform(x_min, x_max, [n_ic,])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.concatenate([np.reshape(xv_ic,[-1,1]), np.reshape(tv_ic,[-1,1])],axis=1)
                       
print(xt_re.shape)
print(xt_bc.shape)
print(xt_ic.shape)

np.savez(
    "data/uniform_6000.npz",
    xt_bc=xt_bc,
    xt_ic=xt_ic,
    xt_re=xt_re
)

(6000, 2)
(200, 2)
(100, 2)


## Grid Mesh Sampling

In [19]:

n_bc = 200  # number of boundary pts
n_ic = 100  # number of initial pts
nx, nt = 50, 40  # residual grid resolution (nx*nt = 2000)
t_min, t_max = 0, 1
x_min, x_max = -1, 1

x = np.linspace(x_min, x_max, nx)
t = np.linspace(t_min, t_max, nt)
xv, tv = np.meshgrid(x, t)
xt_re = np.stack([xv.ravel(), tv.ravel()], axis=1)

x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2])
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc)
xt_bc = np.stack([xv_bc.ravel(), tv_bc.ravel()], axis=1)

x_ic = np.random.uniform(x_min, x_max, [n_ic])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.stack([xv_ic.ravel(), tv_ic.ravel()], axis=1)

print(xt_re.shape)  # (2000, 2)
print(xt_bc.shape)  # (200, 2)
print(xt_ic.shape)  # (100, 2)

np.savez("data/uniform_mesh_2000.npz", xt_bc=xt_bc, xt_ic=xt_ic, xt_re=xt_re)

(2000, 2)
(200, 2)
(100, 2)


In [20]:
import numpy as np

n_bc = 200  # number of boundary pts
n_ic = 100  # number of initial pts
nx, nt = 100, 40  # residual grid resolution (nx*nt = 4000)
t_min, t_max = 0, 1
x_min, x_max = -1, 1

# === Residual points (interior grid) ===
x = np.linspace(x_min, x_max, nx)
t = np.linspace(t_min, t_max, nt)
xv, tv = np.meshgrid(x, t)
xt_re = np.stack([xv.ravel(), tv.ravel()], axis=1)

# === Boundary points ===
x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2])
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc)
xt_bc = np.stack([xv_bc.ravel(), tv_bc.ravel()], axis=1)

# === Initial condition points ===
x_ic = np.random.uniform(x_min, x_max, [n_ic])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.stack([xv_ic.ravel(), tv_ic.ravel()], axis=1)

print(xt_re.shape)  # (2000, 2)
print(xt_bc.shape)  # (200, 2)
print(xt_ic.shape)  # (100, 2)

# === Save all to one file ===
np.savez("data/uniform_mesh_4000.npz", xt_bc=xt_bc, xt_ic=xt_ic, xt_re=xt_re)


(4000, 2)
(200, 2)
(100, 2)


In [21]:
import numpy as np

n_bc = 200  # number of boundary pts
n_ic = 100  # number of initial pts
nx, nt = 100, 60  # residual grid resolution (nx*nt = 6000)
t_min, t_max = 0, 1
x_min, x_max = -1, 1

# === Residual points (interior grid) ===
x = np.linspace(x_min, x_max, nx)
t = np.linspace(t_min, t_max, nt)
xv, tv = np.meshgrid(x, t)
xt_re = np.stack([xv.ravel(), tv.ravel()], axis=1)

# === Boundary points ===
x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2])
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc)
xt_bc = np.stack([xv_bc.ravel(), tv_bc.ravel()], axis=1)

# === Initial condition points ===
x_ic = np.random.uniform(x_min, x_max, [n_ic])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.stack([xv_ic.ravel(), tv_ic.ravel()], axis=1)

print(xt_re.shape)  # (2000, 2)
print(xt_bc.shape)  # (200, 2)
print(xt_ic.shape)  # (100, 2)

# === Save all to one file ===
np.savez("data/uniform_mesh_6000.npz", xt_bc=xt_bc, xt_ic=xt_ic, xt_re=xt_re)


(6000, 2)
(200, 2)
(100, 2)


## LHC Sampling

In [ ]:
from scipy.stats.qmc import LatinHypercube

n_bc = 200 # number of boundary pts
n_ic = 100 # num of init pts
n_re = 2000 # num residual pts
t_min, t_max = 0, 1
x_min, x_max = -1,1


x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2, ]) # two boundary halves
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc) # generate coords on boundaries
xt_bc = np.concatenate([np.reshape(xv_bc, [-1,1]), np.reshape(tv_bc, [-1,1])], axis = 1) # reshape to [(xi, ti)]


x_ic = np.random.uniform(x_min, x_max, [n_ic,])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.concatenate([np.reshape(xv_ic,[-1,1]), np.reshape(tv_ic,[-1,1])],axis=1)

sampler = qmc.LatinHypercube(d=2)
X_lhs = sampler.random(n=n_re) 

# need to rescale to right domain
mins = np.array([x_min, t_min])
maxs = np.array([x_max, t_max])
xt_re = X_lhs * (maxs - mins) + mins

print(xt_re.shape)   # (2000, 2)
print(xt_re.min(axis=0), xt_re.max(axis=0))  # should be close to [-1, 0], [1, 1]


print(xt_re.shape)
print(xt_bc.shape)
print(xt_ic.shape)

np.savez(
    "data/LHS_2000.npz",
    xt_bc=xt_bc,
    xt_ic=xt_ic,
    xt_re=xt_re
)



(2000, 2)
[-9.99940968e-01  1.97452496e-04] [0.9997562  0.99965217]
(2000, 2)
(200, 2)
(100, 2)
(2000, 2)
[-9.99087944e-01  2.46821638e-04] [0.99964158 0.99999128]


In [26]:
from scipy.stats.qmc import LatinHypercube

n_bc = 200 # number of boundary pts
n_ic = 100 # num of init pts
n_re = 4000 # num residual pts
t_min, t_max = 0, 1
x_min, x_max = -1,1


x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2, ]) # two boundary halves
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc) # generate coords on boundaries
xt_bc = np.concatenate([np.reshape(xv_bc, [-1,1]), np.reshape(tv_bc, [-1,1])], axis = 1) # reshape to [(xi, ti)]


x_ic = np.random.uniform(x_min, x_max, [n_ic,])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.concatenate([np.reshape(xv_ic,[-1,1]), np.reshape(tv_ic,[-1,1])],axis=1)

sampler = qmc.LatinHypercube(d=2)
X_lhs = sampler.random(n=n_re) 

# need to rescale to right domain
mins = np.array([x_min, t_min])
maxs = np.array([x_max, t_max])
xt_re = X_lhs * (maxs - mins) + mins

print(xt_re.shape)   # (2000, 2)
print(xt_re.min(axis=0), xt_re.max(axis=0))  # should be close to [-1, 0], [1, 1]


print(xt_re.shape)
print(xt_bc.shape)
print(xt_ic.shape)

np.savez(
    "data/LHS_4000.npz",
    xt_bc=xt_bc,
    xt_ic=xt_ic,
    xt_re=xt_re
)


(4000, 2)
[-9.99973612e-01  2.27273531e-04] [0.99956007 0.99975389]
(4000, 2)
(200, 2)
(100, 2)


In [27]:
from scipy.stats.qmc import LatinHypercube

n_bc = 200 # number of boundary pts
n_ic = 100 # num of init pts
n_re = 6000 # num residual pts
t_min, t_max = 0, 1
x_min, x_max = -1,1


x_bc = np.array([-1.0, 1.0])
t_bc = np.random.uniform(0.0, t_max, [n_bc // 2, ]) # two boundary halves
xv_bc, tv_bc = np.meshgrid(x_bc, t_bc) # generate coords on boundaries
xt_bc = np.concatenate([np.reshape(xv_bc, [-1,1]), np.reshape(tv_bc, [-1,1])], axis = 1) # reshape to [(xi, ti)]


x_ic = np.random.uniform(x_min, x_max, [n_ic,])
t_ic = np.array([0.0])
xv_ic, tv_ic = np.meshgrid(x_ic, t_ic)
xt_ic = np.concatenate([np.reshape(xv_ic,[-1,1]), np.reshape(tv_ic,[-1,1])],axis=1)

sampler = qmc.LatinHypercube(d=2)
X_lhs = sampler.random(n=n_re) 

# need to rescale to right domain
mins = np.array([x_min, t_min])
maxs = np.array([x_max, t_max])
xt_re = X_lhs * (maxs - mins) + mins

print(xt_re.shape)   # (2000, 2)
print(xt_re.min(axis=0), xt_re.max(axis=0))  # should be close to [-1, 0], [1, 1]


print(xt_re.shape)
print(xt_bc.shape)
print(xt_ic.shape)

np.savez(
    "data/LHS_6000.npz",
    xt_bc=xt_bc,
    xt_ic=xt_ic,
    xt_re=xt_re
)


(6000, 2)
[-9.99816129e-01  6.56921950e-05] [0.9998951  0.99988922]
(6000, 2)
(200, 2)
(100, 2)
